![](pictures/shopping_propensity.png)

<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
Ноутбук от 30.09.23 
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>   
**_"Data Science Analyst"_  
_МОДЕЛЬ СКЛОННОСТИ КЛИЕНТОВ К ПОКУПКАМ В ФЕДЕРАЛЬНОЙ СЕТИ ПРОДАЖ СПОРТИВНЫХ ТОВАРОВ_**  

Выполнено по технологии CRISP-DM:  
Аргентов Сергей, г.Липецк  
https://argentov.pro
<b>  </b>  
<b>  </b>  

<h1>СОДЕРЖАНИЕ<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Предобработка-данных" data-toc-modified-id="1.-Предобработка-данных-1">1. Предобработка данных</a></span></li><li><span><a href="#2.-Бинарная-классификация" data-toc-modified-id="2.-Бинарная-классификация-2">2. Бинарная классификация</a></span></li><li><span><a href="#3.-Анализ-проведённого-АВ-тестирования" data-toc-modified-id="3.-Анализ-проведённого-АВ-тестирования-3">3. Анализ проведённого АВ-тестирования</a></span></li><li><span><a href="#4.-Кластерезация" data-toc-modified-id="4.-Кластерезация-4">4. Кластерезация</a></span></li><li><span><a href="#5.-Построение-модели-склорнности-клиентов-к-покупкам" data-toc-modified-id="5.-Построение-модели-склорнности-клиентов-к-покупкам-5">5. Построение модели склорнности клиентов к покупкам</a></span></li></ul></div>

# 1. Предобработка данных

# 2. Бинарная классификация

# 3. Анализ проведённого АВ-тестирования

# 4. Кластерезация

# 5. Построение модели склорнности клиентов к покупкам